In [1]:
from gensim.models import word2vec,Word2Vec
from gensim.utils import tokenize
import pandas as pd
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize

In [22]:
test = pd.read_json("test-data.json","r", encoding="utf8")
train = pd.read_json("train-data.json","r", encoding="utf8")

In [23]:
df = pd.concat([test,train])

In [24]:
df.head()

,_id,corrected_question,intermediary_question,sparql_query,sparql_template_id
0,1701,Which architect of Marine Corps Air Station Ka...,What is the <architect> of the <Marine Corps A...,SELECT DISTINCT ?uri WHERE { <http://dbpedia....,16
1,3293,Which sects people are part of local political...,What is the <faith> of the <political parties>...,SELECT DISTINCT ?uri WHERE { ?x <http://dbpedi...,305
2,2161,List common systems are developed by of the Go...,What is the <developed by> of the <Google Vide...,SELECT DISTINCT ?uri WHERE { <http://dbpedia....,16
3,1136,For which games are Sam Loyd and Eric Schiller...,What is the <known for> of the <Sam Loyd> and ...,SELECT DISTINCT ?uri WHERE { <http://dbpedia.o...,15
4,987,Is Peter Piper Pizza in the pizza industry?,Is <Peter Piper Pizza> the <field of exercise>...,ASK WHERE { <http://dbpedia.org/resource/Peter...,152


In [25]:
df.shape

(5000, 5)

In [26]:
test.shape

(1000, 5)

In [27]:
def preprocess_label(review):
    review_text = review
    words = review_text.split()
    return (words)
def labels_to_sentences(label):
    raw_sentences = sent_tokenize(label.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(preprocess_label(raw_sentence))
    return sentences

In [30]:
sentences = []
for x in df.sparql_query.apply(str):
    sentences += labels_to_sentences(x)
len(sentences)    

10162

In [31]:
num_features = 300  
min_word_count = 1
num_workers = 4     
context = 10       
downsampling = 1e-3 # (0.001) 

from gensim.models import word2vec
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling,
                          sg = 1
                         )

model.init_sims(replace=True)

In [32]:
model.wv.vectors.shape

(4772, 300)

In [38]:
model.wv.most_similar("<http://dbpedia.org/ontology/Musical>}")

[('<http://dbpedia.org/ontology/VideoGame>}', 0.8688365817070007),
 ('<http://dbpedia.org/ontology/Stadium>}', 0.8639969825744629),
 ('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', 0.8630322217941284),
 ('<http://dbpedia.org/ontology/Software>}', 0.8386920094490051),
 ('<http://dbpedia.org/ontology/MilitaryUnit>}', 0.8346012830734253),
 ('<http://dbpedia.org/ontology/SoccerClubSeason>}', 0.81864333152771),
 ('<http://dbpedia.org/ontology/BroadcastNetwork>}', 0.8127313852310181),
 ('<http://dbpedia.org/ontology/EducationalInstitution>}', 0.8118859529495239),
 ('<http://dbpedia.org/ontology/Building>}', 0.8009661436080933),
 ('<http://dbpedia.org/ontology/Politician>}', 0.7992528080940247)]

In [36]:
model.wv.most_similar("WHERE")

[('DISTINCT', 0.9657090306282043),
 ('COUNT(?uri)', 0.9650171995162964),
 ('.', 0.964745044708252),
 ('?uri.', 0.9607105851173401),
 ('}', 0.9601176381111145),
 ('?x', 0.9488901495933533),
 ('?uri', 0.9366395473480225),
 ('{', 0.9314072132110596),
 ('{?uri', 0.9292231798171997),
 ('SELECT', 0.9277162551879883)]

In [37]:
model.wv.most_similar("SELECT")

[('.', 0.9721692204475403),
 ('COUNT(?uri)', 0.968413233757019),
 ('?uri.', 0.9658728837966919),
 ('}', 0.9658405184745789),
 ('{', 0.958430290222168),
 ('?x', 0.9583249688148499),
 ('DISTINCT', 0.9575579762458801),
 ('{?uri', 0.9532997608184814),
 ('ASK', 0.9387346506118774),
 ('?uri', 0.9384315013885498)]